<a href="https://colab.research.google.com/github/427Stanley427/Python/blob/main/G%C3%A9n%C3%A9rer%20une%20paire%20de%20cl%C3%A9%20bitcoin%20al%C3%A9atoire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install bitcoin

In [4]:
pip install requests

In [ ]:
import requests
import bitcoin

def generate_bitcoin_keypair():
    private_key = bitcoin.random_key()
    public_key_uncompressed = bitcoin.privtopub(private_key)
    public_key_compressed = bitcoin.compress(bitcoin.decode_pubkey(public_key_uncompressed, 'hex'))
    address_uncompressed = bitcoin.pubtoaddr(public_key_uncompressed)
    address_compressed = bitcoin.pubtoaddr(public_key_compressed)

    wif_private_key = bitcoin.encode_privkey(bitcoin.decode_privkey(private_key, 'hex'), 'wif')

    return {
        "private_key_hex": private_key,
        "wif_private_key": wif_private_key,
        "public_key_uncompressed": public_key_uncompressed,
        "public_key_compressed": public_key_compressed,
        "address_uncompressed": address_uncompressed,
        "address_compressed": address_compressed
    }

def get_balance(address):
    url = f"https://api.blockchair.com/bitcoin/dashboards/address/{address}"
    response = requests.get(url)
    data = response.json().get("data", {})

    if not data:
        return 0

    address_data = data.get(address, {})
    if not address_data:
        return 0

    balance_satoshis = address_data.get("address", {}).get("balance", 0)
    return balance_satoshis / 100_000_000  # Convertir en BTC

kp = generate_bitcoin_keypair()
balance_uncompressed = get_balance(kp["address_uncompressed"])
balance_compressed = get_balance(kp["address_compressed"])

print("Clé privée hex :", kp["private_key_hex"])
print("Clé privée WIF :", kp["wif_private_key"])
print("Clé publique non compressée :", kp["public_key_uncompressed"])
print("Clé publique compressée :", kp["public_key_compressed"])
print("Adresse non compressée :", kp["address_uncompressed"], "| Balance:", balance_uncompressed, "BTC")
print("Adresse compressée :", kp["address_compressed"], "| Balance:", balance_compressed, "BTC")
